# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm:  RSA Encryption Scheme

In [1]:
import time

In [2]:
from os import urandom
from hashlib import sha512

# Algorithm

In [3]:
%%time
def rsa_generate_keys():
    return [
        65537,
        58967658369561163583995664151705537612631456941226585145001736155445085885436956133402962616775555500479429922140321605063456075222335023020218578571558003435174909963319619244821157746252197885628802071763470174413201522569356053296685834595362968800778468737693074613267684084217204017873750446802044584084498581219849973790017343888256411013653688556278788070745635045095995056877259642839730825907965544973672656542601570609068817838234644958846427643088478240335082249677864789882511592486797239674160452077169411971273434857626735582274817190984442183721945999865859466422472845277588368259261760233826535480137
    ], [
        32639742054323523661031580828650534544392003478949839063736255562124081596351847364013089886417596950354636310108218358259943735367279937975211699593540109138569129405212055903155962561652878992005591100527818545966603574053221236696683939389678915058929150433015761702105657992264877747720954135956649973789334911071168428227464085150820871588160770978551544646965210798269197906675922224772713666123225990305644372957419486169245295190574189157389340237417783311258488777336686103120891002317113842264416737708675921812070527474901946450952078789439410581693777829144977217172397092723130874770379072485175449578961,
        58967658369561163583995664151705537612631456941226585145001736155445085885436956133402962616775555500479429922140321605063456075222335023020218578571558003435174909963319619244821157746252197885628802071763470174413201522569356053296685834595362968800778468737693074613267684084217204017873750446802044584084498581219849973790017343888256411013653688556278788070745635045095995056877259642839730825907965544973672656542601570609068817838234644958846427643088478240335082249677864789882511592486797239674160452077169411971273434857626735582274817190984442183721945999865859466422472845277588368259261760233826535480137
    ]

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.58 µs


In [4]:
%%time
def bxor(x, y):
    return bytes(i ^ j for i, j in zip(x, y))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.3 µs


In [5]:
%%time
def rsa_encrypt(plaintext, public_key):
    # iv[64] -> h1[64] -> h2[64] -> h3[64]
    iv = urandom(64)
    h1 = sha512(iv).digest()
    h2 = sha512(h1).digest()
    h3 = sha512(h2).digest()

    # x[192] := pt[192] ^ (h1|h2|h3)[192]
    pt = int.to_bytes(plaintext, 192, 'big')
    x192 = bxor(pt, h1 + h2 + h3)

    # x[64] := iv[64] ^ x[192->64]
    h4 = sha512(x192).digest()
    x64 = bxor(iv, h4)

    # x[256] := x[192]|x[64]
    x256 = int.from_bytes(x192 + x64, 'big')

    # rsa
    return pow(x256, *public_key)

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 12.9 µs


In [6]:
%%time
def rsa_decrypt(ciphertext, secret_key):
    # rsa
    x256 = pow(ciphertext, *secret_key)

    # x[192]|x[64] := x[256]
    x256 = int.to_bytes(x256, 256, 'big')
    x192, x64 = x256[:192], x256[192:]

    # iv[64] := x[64] ^ x[192->64]
    h4 = sha512(x192).digest()
    iv = bxor(x64, h4)

    # iv[64] -> h1[64] -> h2[64] -> h3[64]
    h1 = sha512(iv).digest()
    h2 = sha512(h1).digest()
    h3 = sha512(h2).digest()

    # pt[192] := x[192] ^ (h1|h2|h3)[192]
    pt = bxor(x192, h1 + h2 + h3)

    # plaintext
    return int.from_bytes(pt, 'big')

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.3 µs


# Run

## RSA Keys

In [7]:
%%time
public_key, secret_key = rsa_generate_keys()

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.78 µs


## Dummy Message #1

In [8]:
%%time
ciphertext = rsa_encrypt(0, public_key)
plaintext = rsa_decrypt(ciphertext, secret_key)
assert plaintext == 0
ciphertext

CPU times: user 42.4 ms, sys: 0 ns, total: 42.4 ms
Wall time: 49.4 ms


31906251898968821735256128397574959835041519070098253276843399376489768355911574020941155617885990542691132337975531746474282091769223958171572498125277565334733669952272491816684348230267213769555788614363061735971619419868691532126088569977924505132724010111659303048231204764468045295498297911320772658080661281062228194375469059530954808408557043938418987187919800026878698593005670466567201266790914895799472512501789409444757585600574621510118392980581136770231179828462976438826244730597835646739840602793013057843462819313149234495075147452160785805217914853987399144566323653434573260883380359383852029841514

## Dummy Message #2

In [9]:
%%time
ciphertext = rsa_encrypt(0, public_key)
plaintext = rsa_decrypt(ciphertext, secret_key)
assert plaintext == 0
ciphertext

CPU times: user 38.9 ms, sys: 1.43 ms, total: 40.3 ms
Wall time: 41.1 ms


33021773656312827541453343971317568085324416664258739002952423791912728402102788553573343663393241294195547433454497697320839242206238040966657045996220936750285202475109172521638332560738054972194956178333642196122546264732567395685019380668951449118059651703230748951313819644112497020367810967505269881746849148218967246159386496696955949087533504466288186870542077094536036207704375963050491038782535710457368248597801320485931058503517836829857450549558690198975435381170835218988381280453613174588381894018360991705759983409580602780777761344634747963697300899446731031863414900659709331997061893975292089385833

# The End